#### Understanding the Problem 
        Customer reviews and application are longer and descriptive. if we check manually then it will take much longer time, so here comes the excellence of the Natural language processing, i'm building the text summarizer. 
    

#### Importing the Necessary Libraries

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pylab as plt 
import seaborn as sns 

from bs4 import BeautifulSoup

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Embedding, TimeDistributed, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

#### Reading the Dataset

In [ ]:
dataset = pd.read_csv('dataset\Reviews.csv')
dataset = dataset[['Summary', 'Text']]
dataset.head()

In [ ]:
dataset.isna().sum()
dataset.dropna(axis=0, inplace=True)
dataset.drop_duplicates(keep='first')

In [ ]:
dataset = dataset.rename(columns={'Summary': 'summary', 'Text': 'text'})
dataset.head()

In [ ]:
len_dataset = len(dataset)
print(f"length of the dataset : {len_dataset}")

#### Dataset Preprocessing

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                       "didn't": "did not", "doesn't": "does not", "don't": "do not",

                        "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did", 
                        "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                       
                       "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "i'd": "i would",
                       "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have", "i'm": "i am", "i've": "i have",
                        
                        "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam",
                       
                       "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have",
                       "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", 
                       
                       "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                       "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                       "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so as",
                       "this's": "this is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                       "there'd've": "there would have", "there's": "there is", "here's": "here is", "they'd": "they would", "they'd've": "they would have",
                       "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                       "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                       "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                       "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                       "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                       "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                       "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                       "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                       "you're": "you are", "you've": "you have"}

In [ ]:
stop_words = stopwords.words('english')

def text_preprocessing(text):
    text = text.lower()
    text = BeautifulSoup(text, 'lxml').text # removing html tags
    text = re.sub("'s", " ", text)    # it matches with only ( 's )
    text = re.sub("\([^)]*\)", " ", text) # replaces the () anything inside with white spaces 
    text = re.sub('"', '', text)
    text = re.sub("^\w\s", '', text)
    text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    text = [t for t in text.split(' ') if t not in stop_words]
    final_text = []
    for t in text:
        if len(t) >= 3:
            final_text.append(t)

    final_text = ' '.join(final_text).strip()
    return final_text
        

In [ ]:
def summary_preprocessing(text):
    newString = re.sub('"', '', str(text))  # remove quotes
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])  # the dictionary that we will use for expanding the contractions
    newString = re.sub(r"'s", "", newString)    # removing the matches with ('s)
    text = re.sub("\([^)]*\)", " ", text) # removing the words inside like (shiva)
    newString = re.sub("[^a-zA-Z]", " ", newString)   # match all strings that contain a non-letter
    newString = newString.lower()  # lowercase words
    tokens = newString.split()
    newString = ''
    for i in tokens:
        if len(i) > 1:
            newString = newString+i+' '
    return newString

In [ ]:
dataset['cleaned_text'] = dataset['text'].apply(text_preprocessing)
dataset['cleaned_summary'] = dataset['summary'].apply(summary_preprocessing)
dataset.head()

In [ ]:
def count_characters(text):
    icount = 0
    for word in text:
        if word.isdigit():
            continue
        icount = icount + 1
    return icount

In [ ]:
dataset['word_count_text'] = dataset['cleaned_text'].apply(lambda x : len(word_tokenize(x)))
dataset['word_count_summary'] = dataset['cleaned_summary'].apply(lambda x: len(word_tokenize(x)))
dataset['char_count_text'] = dataset['cleaned_text'].apply(count_characters)
dataset['char_count_summary'] = dataset['cleaned_summary'].apply(count_characters)

In [ ]:
dataset.head()

In [ ]:
print(max(dataset['word_count_text']))
print(max(dataset['word_count_summary']))
print(max(dataset['char_count_text']))
print(max(dataset['char_count_summary']))

In [ ]:
# handaling distributions word counts 
plt.figure(figsize=(10, 7))
sns.histplot(dataset['char_count_text'], kde=True, label='char_count_text', color='blue')
sns.histplot(dataset['char_count_summary'], kde=True, label='char_count_summary', color='red')

plt.xscale('log')
plt.title('Distributions of Word Count Text VS Summary', fontsize=12, fontweight='bold')
plt.xlabel('Blue - FrequencyOfText / Red - FrequencyOfSummary', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
plt.legend()
plt.show()

In [ ]:
# handaling distributions word counts 
plt.figure(figsize=(10, 7))
sns.histplot(dataset['word_count_text'], kde=True, label='word_count_text', color='blue')
sns.histplot(dataset['word_count_summary'], kde=True, label='word_count_summary', color='red')

plt.xlim(0, 100)
plt.title('Distributions of Word Count Text VS Summary', fontsize=12, fontweight='bold')
plt.xlabel('Blue - FrequencyOfText / Red - FrequencyOfSummary', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
plt.legend()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['cleaned_text'], dataset['cleaned_summary'], test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)


print(y_test.shape)


In [ ]:
print(len(X_train), type(X_train))

In [ ]:
max_len_text = 80
max_len_summary = 20

In [ ]:
# Preparing tokenizer
X_tokenizer = Tokenizer()
X_tokenizer.fit_on_texts(list(X_train))

X_train = X_tokenizer.texts_to_sequences(X_train)
X_test = X_tokenizer.texts_to_sequences(X_test)

# padding zero upto the maximum length
X_train = pad_sequences(X_train, maxlen=max_len_text, padding='post')
X_test = pad_sequences(X_train, maxlen=max_len_text, padding='post')

X_voc_size = len(X_tokenizer.word_index) + 1


In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
len(X_tokenizer.word_index)

In [ ]:
# summary tokenizer
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_train))

y_train = y_tokenizer.texts_to_sequences(y_train)
y_test = y_tokenizer.texts_to_sequences(y_test)

# padding zero upto the maximum length
y_train = pad_sequences(y_train, maxlen=max_len_summary, padding='post')
y_test = pad_sequences(y_test, maxlen=max_len_summary, padding='post')

y_voc_size = len(y_tokenizer.word_index) + 1

In [ ]:
y_train.shape

In [ ]:
len(y_tokenizer.word_index)

In [ ]:
latent_dim = 500

# Encoder 
encoder_inputs = Input(shape=(max_len_text, ))
# Word embeddings provide dense representation or words and their relative meanings, each input integer
# is used as index to access a table that contains all possible vectors. That is the reason why you need 
# to specify the size of the vocabulary as the first argument (so the table can be initialized). 

encoder_embedding = Embedding(X_voc_size, latent_dim, trainable=True)(encoder_inputs)

# LSTM 1 
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_embedding)

# LSTM 2
encoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# LSTM 3 
encoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True)
enccoder_output3, state_h, state_c = encoder_lstm3(encoder_output2)

# Set up the decoder 
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(y_voc_size, latent_dim, trainable=True)
document_embdedding = dec_emb_layer(decoder_inputs)

# Lstm using encoder_states 
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_curr, decoder_cell = decoder_lstm(document_embdedding, initial_state = [state_h, state_c])

# Dense Layer 
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()


In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1)


history = model.fit([X_train, y_train[:, :-1]], y_train.reshape(y_train.shape[0], y_train.shape[1], 1)[:, 1:], epochs=20,
                    batch_size=512, validation_data=([X_test, y_test[:, :-1]], y_test.reshape(y_test.shape[0], y_test.shape[1], 1)[:, 1:]))

In [ ]:
from matplotlib import pyplot 
pyplot.plot(history.history['loss'], label='train') 
pyplot.plot(history.history['val_loss'], label='test') 
pyplot.legend()
pyplot.show()

In [ ]:
reverse_target_word_index = y_tokenizer.index_word 
reverse_source_word_index = X_tokenizer.index_word 
target_word_index = y_tokenizer.word_index

In [ ]:
# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# attention inference
# attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
# decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c], [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_len_summary-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    '''
    convert an integer sequence to a word sequence for summary
    '''
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    '''
    convert an integer sequence to a word sequence for reviews
    '''
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
for i in range(20, 30):
  print("Review:", seq2text(X_test[i]))
  print("Original summary:", seq2summary(y_test[i]))
  print("Predicted summary:", decode_sequence(X_test[i].reshape(1, max_len_text)))
  print("\n")